# Introduction to data analysis for natural and social sciences
The present notebook contitutes the first part of the exam. Portions of data analysis and simulations present in article "Patient-specific Boolean models of signalling networks guide personalised treatments" are reproduced.

Notebook content is based on version 2 of the article, published April 18, 2022 (DOI: https://doi.org/10.7554/eLife.72626).

## Imports and global settings
Softwares bioLQM and GINsim are used in the notebook to perform operations on the Boolean model. They are accessed through their respective Python interfaces.

In [2]:
import biolqm
import ginsim
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

PATH_REPORT = "report"

EXT_EXCEL = "xlsx"
EXT_TAB = "tsv"

This notebook has been executed using the docker image `colomoto/colomoto-docker:2023-06-01`

# Prostate Boolean model construction
The prostate Boolean model is built starting from information available in existing literature. A generic cancer Boolean network is used as baseline and contains information on pathways and interactions between nodes of the underlying signalling network which are commonly found in turmors.

Nodes of the network are biological entities involved in the regulation of the functions of the cell, e.g. genes, proteins, environmental conditions, expression factors. Associated to nodes there are information such as logical rules governing the dynamics of the Boolean network and the pathway each node belongs to.

Edges represent interactions between the biological entities. They can be positive, when the activation of the source node results in the activation of the target node, or negative, when the activation of the source node inhibites the target node.

The initial model is then enriched with prostate cancer-specific information. Software ROMA is used to identify new pathways from transcriptomics and proteomics data, by selecting highly variable genes for each pathway. Transcriptomics data is gathered from The Cancer Genome Atlas (TCGA) selecting gene sets pertained to cancer pathways. Database OmniPath is used to establish connections between genes present in the model and proteins known to have a function in prostate or found with ROMA.

The final network consists of 133 nodes and 449 edges. It is then translated to a Boolean model by identifying input nodes, which represent physiological conditions of the cell, and output nodes, which represent phenotypes of normal cell processes or cancer development.

## Data exploration
In this section, some operations are performed on the network data to develop a better understanding about their organisation and meaning.

The authors collected all data in the following Excel files. Data are loaded in Pandas dataframes to make their manipulation easier. Dataframe `df_nodes_pathways` relates pathways to nodes, dataframe `df_nodes_interactions` contains information about edges and nodes, e.g. logical rules, references to literature, interaction types, and dataframe `df_nodes_unique` lists additional information about nodes.

In [18]:
fname_nodes_pathways = "Montagud2022_nodes in pathways.xlsx"
fname_nodes_network = "Montagud2022_interactions_sources.xlsx"

df_nodes_pathways = pd.read_excel(
    io=f"{PATH_REPORT}/{fname_nodes_pathways}",
    header=None,
    names=["node", "pathway"]
)
print("Dataframe df_nodes_pathways")
df_nodes_pathways.info(verbose=True, memory_usage=False)

sheet_interactions = "Nodes"
df_nodes_interactions = pd.read_excel(
    io=f"{PATH_REPORT}/{fname_nodes_network}",
    sheet_name=sheet_interactions,
    header=1,
    converters={"Reference: PMID": lambda c: np.str_(c).strip()}  # Remove useless line breaks in cells.
)
print("\n\nDataframe df_nodes_interactions")
df_nodes_interactions.info(verbose=True, memory_usage=False)

sheet_unique="Nodes_unique"
df_nodes_unique = pd.read_excel(
    io=f"{PATH_REPORT}/{fname_nodes_network}",
    sheet_name=sheet_unique
)
print("\n\nDataframe df_nodes_unique")
df_nodes_unique.info(verbose=True, memory_usage=False)

Dataframe df_nodes_pathways
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133 entries, 0 to 132
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   node     133 non-null    object
 1   pathway  133 non-null    object
dtypes: object(2)

Dataframe df_nodes_interactions
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 462 entries, 0 to 461
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Target node       462 non-null    object
 1   HUGO names        454 non-null    object
 2   Interaction type  462 non-null    object
 3   Source            462 non-null    object
 4   Description       462 non-null    object
 5   Reference: PMID   453 non-null    object
 6   Logical rule      462 non-null    object
dtypes: object(7)

Dataframe df_nodes_unique
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121 entries, 0 to 120
Data columns (total 4 columns):
 #

### Check consistency
A single logical rule is associated to each node. Accordingly, the result of grouping interaction data by node and logical rule is a list of 133 rows.

Moreover, the number of edges can be obtained by counting pairs of source node and target node, excluding input nodes as targets.

In [23]:
display(df_nodes_interactions.groupby(["Target node", "Logical rule"]).count())

df_edge_no_inputs = df_nodes_interactions.loc[df_nodes_interactions["Source"] != "0/1"]
display(df_edge_no_inputs.groupby(["Target node", "Source"]).count())
del df_edge_no_inputs

,,HUGO names,Interaction type,Source,Description,Reference: PMID
Target node,Logical rule,,,,,
AKT,((HSPs | (PDK1 & PIP3) | PIP3 | (SHH & PIP3)) & !PTCH1),5,5,5,5,5
AMPK,(ATR | HIF1 | AMP_ATP | ATM) & !FGFR3,6,6,6,6,6
AMP_ATP,(!Nutrients),1,1,1,1,1
APAF1,((Caspase8 | BAX | p53 | Bak | HSPs) & !Bcl_XL & !BCL2 & !AKT),8,8,8,8,8
AR,((GLI | EP300 | HSPs | NKX3_1 | EZH2 | NCOA3 | PKC | SMAD | Androgen) & !PTEN & !NCOR1 & !NCOR2 & !MDM2),13,13,13,13,13
...,...,...,...,...,...,...
p21,((p53 | SMAD | HIF1 | ZBTB17) & !TERT & !MYC_MAX & !MDM2 & !AKT & !ERK),9,9,9,9,9
p38,(MAP3K1_3 & !ERK & !GADD45),3,3,3,3,3
p53,((Acidosis | CHK1_2 | p38 | HIF1) & !BCL2 & !MDM2 & !HSPs & !Snail),9,9,9,9,9


HUGO names  Interaction type  Description  \
Target node Source                                              
AKT         HSPs             1                 1            1   
            PDK1             1                 1            1   
            PIP3             1                 1            1   
            PTCH1            1                 1            1   
            SHH              1                 1            1   
...                        ...               ...          ...   
p53         p38              1                 1            1   
p70S6kab    PDK1             1                 1            1   
            mTORC2           1                 1            1   
p90RSK      ERK              1                 1            1   
            PDK1             1                 1            1   

                    Reference: PMID  Logical rule  
Target node Source                                 
AKT         HSPs                  1             1  
            PDK1                  1             1  
            PIP3                  1             1  
            PTCH1                 1             1  
            SHH                   1             1  
...                             ...           ...  
p53         p38                   1             1  
p70S6kab    PDK1                  1             1  
            mTORC2                1             1  
p90RSK      ERK                   1             1  
            PDK1                  1             1  

[449 rows x 5 columns]

In [64]:
# MC debug, remove in v0.2.0
# df_nodes_unique contains only 121 nodes and not 133 as df_nodes_pathways.
# In other words, df_nodes_unique["Node"] should be a subset of df_nodes_pathways["node"].
# Why is that? Let's find out.

df = df_nodes_pathways.set_index("node")
df_subset = df_nodes_unique.set_index("Node")

# The input nodes should be all removed since they are not regulated (by authors' design),
# hence they are not part of any pathway. But this is not what happens.
display(df.loc[df["pathway"] == "Input"])
display(df.drop(labels=df_subset.index, errors="ignore"))

# Moreover, in df_nodes_unique there is a node called MAX which is not part of the nodes considered for the final network.
# Surprisingly, it is not present among the nodes in df_nodes_interactions.
try:
    df.drop(labels=df_subset.index)
except Exception as e:
    print(e)
finally:
    del df
    del df_subset

# In conclusion, it seems that the choice of nodes from the Excel files can not be deduced directly
# just from the observation of the content of the files.
# In particular, I should use data in df_nodes_unique with caution,
# since their relation with the other data is not straightforward.

### Data export
Data about nodes are exported in files with tab-separated values (TSV) format. Exported data can be imported in software Cytoscape for later use.

In [65]:
name_nodes_pathways = fname_nodes_pathways.removesuffix(f".{EXT_EXCEL}")
name_nodes_network = fname_nodes_network.removesuffix(f".{EXT_EXCEL}")

df_nodes_pathways.to_csv(
    path_or_buf=f"{name_nodes_pathways}.{EXT_TAB}",
    sep='\t',
    index=False
)
df_nodes_interactions.to_csv(
    path_or_buf=f"{name_nodes_network}_{sheet_interactions}.{EXT_TAB}",
    sep='\t',
    index=False
)
df_nodes_unique.to_csv(
    path_or_buf=f"{name_nodes_network}_{sheet_unique}.{EXT_TAB}",
    sep='\t',
    index=False
)

Mandatory data to recreate the network are: source nodes, target nodes, interaction types, logical rules and pathways. A single file with at least these data can be constructed.

A note on the use of this file in Cytoscape: node "0/1" should be hidden because it is generated by the software as source node for the input nodes.

In [66]:
df_cytoscape = df_nodes_interactions.join(
    other=df_nodes_pathways.set_index("node"),
    on="Target node"
)
df_cytoscape.to_csv(
    path_or_buf=f"cytoscape_data.{EXT_TAB}",
    sep='\t',
    index=False
)

To build the regulatory network, GINsim is used by the authors. The resulting network is exported as ZGINML file, available in the Supplementary file 1 under the name specified in variable `fname_model`. The ZGINML file is then imported in Cytoscape for visual improvement.

In [67]:
fname_model = "Montagud2022_Prostate_Cancer.zginml"

### Network visualisation with GINsim<a id="sec:network_visualisation_with_ginsim"></a>
The GINsim model can be used directly for some tasks and all the information are contained in the ZGINML file. For instance the whole network can be displayed.

In [69]:
ginsim_model = ginsim.load(f"{PATH_REPORT}/{fname_model}")

ginsim.show(ginsim_model)

### Comparison with reference network
Only nine nodes are referred as proper inputs in the article (cfr. Appendix 1.2.3 and subsection "Boolean model construction" in section "Materials and methods"). The remaining two are "fused_event" and "SPOP".

Node "fused_event" is present to consider the condition of fusion between gene promoter TMPRSS2 and transcription factor ERG and is added manually based on existing literature (cfr. Appendix 1.1.5 and subsection "Prior knowledge network construction" in section "Materials and methods").

Node "SPOP" is present to consider mutations of gene SPOP which are frequent in prostate cancer (cfr. Appendix 1.1.2).

The following are supplied data regarding each of these nodes.

In [68]:
display(df_nodes_interactions.loc[df_nodes_interactions["Target node"] == "fused_event"])
display(df_nodes_interactions.loc[df_nodes_interactions["Source"] == "fused_event"])

display(df_nodes_interactions.loc[df_nodes_interactions["Target node"] == "SPOP"])
display(df_nodes_interactions.loc[df_nodes_interactions["Source"] == "SPOP"])

### Variation from reference for interactions of node "SPOP"
Inspecting imported data, node "SPOP" turns out to be an inhibitor for only three of its four children nodes (i.e. negative interaction type), but in the reference network all four interactions are set as inhibiting, following the respective logical rules.

This difference can be seen comparing the nodes from the Excel files which have "SPOP" as source node and the plot of the network loaded from the ZGINML file, made for example with GINsim (cfr. notebook section [Network visualisation with GINsim](#sec:network_visualisation_with_ginsim)).

Logical rules show the correct behaviour.

In [70]:
pat_old = pd.get_option("display.max_colwidth")
pd.set_option("display.max_colwidth", None)
display(df_nodes_interactions.loc[df_nodes_interactions["Source"] == "SPOP", ("Target node", "Interaction type", "Description")])
pd.set_option("display.max_colwidth", pat_old)
del pat_old

### Stable states evaluation with bioLQM
The stable states of the model can be found exactly. Here this task is performed by bioLQM.

In [71]:
biolqm_model = ginsim.to_biolqm(ginsim_model)
#biolqm_fixpoints = biolqm.fixpoints(biolqm_model)  # Comment out to speed up the execution of the notebook.

# Prostate Boolean model simulation
MC explain breifly what are simulations. How they are done in general: set initial conditions, select the observed output nodes and run the stochastic simulation wiht maboss.

To perform simulations using the Boolean model, MaBoSS is used. First, configurations and information about the model are extracted from the GINsim model, then the number of trajectories and other configurations are set.

In [73]:
maboss_model = ginsim.to_maboss(ginsim_model)

maboss_model.update_parameters(
    max_time=30,
    sample_count=5000
)

## Wild type simulation
Specific configurations can be applied to the network and the resulting model can be used to simulate prostate cells under physiological conditions. This model is called by the authors "wild type model".

Simulations using wild type model set inactive initial state to output nodes, active initial state to some input states as specified in the following examples and random initial state to the remaining nodes.

For the first simulation, input nodes of the network have initial state inactive and only some of the output nodes are chosen as outputs of the simulation.

In [74]:
maboss_wtA_model = maboss_model.copy()

for node in maboss_wtA_model.network:
    maboss_wtA_model.network.set_istate(node, [0.5, 0.5])

inactives_wtA = df_nodes_pathways.loc[
    (df_nodes_pathways["pathway"] == "Input") | (df_nodes_pathways["pathway"] == "Output")
]["node"].tolist()
for node in inactives_wtA:
    maboss_wtA_model.network.set_istate(node, [1, 0])

outputs_wtA = ["Apoptosis", "DNA_Repair", "Metastasis", "Proliferation"]
maboss_wtA_model.network.set_output(outputs_wtA)

The result of the simulation is a plot of trajectories of probabilities of given phenotypes (i.e. the output of the simulation).

In [75]:
maboss_wtA_result = maboss_wtA_model.run()

maboss_wtA_result.get_nodes_probtraj().plot(
    legend=True,
    xlabel="Time (a.u.)",
    ylabel="Node activity (%)"
)
plt.savefig("3A.pdf")

The second simulation has input nodes in list `actives_wtB` active from the start. Moreover node "SPOP" is set with random initial state.

Phenotypes of the second simulation are listed in variable `outputs_wtB` and includes some intermediate nodes other than some output nodes.

In [76]:
maboss_wtB_model = maboss_wtA_model.copy()

actives_wtB = ["EGF", "FGF", "Nutrients", "Androgen"]
for node in actives_wtB:
    maboss_wtB_model.network.set_istate(node, [0, 1])
maboss_wtB_model.network.set_istate("SPOP", [0.5, 0.5])

outputs_wtB = ["Apoptosis", "CyclinD", "E2F1", "Metastasis", "Proliferation", "p53"]
maboss_wtB_model.network.set_output(outputs_wtB)

The trajectory plot of the second simulation is:

In [77]:
maboss_wtB_result = maboss_wtB_model.run()

maboss_wtB_result.get_nodes_probtraj().plot(
    legend=True,
    xlabel="Time (a.u.)",
    ylabel="Node activity (%)"
)
plt.savefig("3B.pdf")

The third simulation is executed with initial state active for the nodes in list `actives_wtC` and the output nodes are the same of the second simulation.

In [78]:
maboss_wtC_model = maboss_wtA_model.copy()

actives_wtC = ["Carcinogen", "Androgen", "TNFalpha", "Acidosis", "Hypoxia"]
for node in actives_wtC:
    maboss_wtC_model.network.set_istate(node, [0, 1])

maboss_wtC_model.network.set_output(outputs_wtB)

The trajectory plot of the third simulation is:

In [79]:
maboss_wtC_result = maboss_wtC_model.run()

maboss_wtC_result.get_nodes_probtraj().plot(
    legend=True,
    xlabel="Time (a.u.)",
    ylabel="Node activity (%)"
)
plt.savefig("3C.pdf")

## Model validation
The model is validated by simulating some known interactions between genes in prostate cells under physiological conditions.

For example the order of events during cell cycle is tested. In this case the initial conditions are active growth factors and nutrients, along with nodes related to the activation of the cell cycle, while the cyclins start inactive. All the other nodes have random initial state. The outputs are list in variable `outputs_cellcycle`.

In [80]:
maboss_cellcycle_model = maboss_wtA_model.copy()

actives_cellcycle = ["EGF", "FGF", "Nutrients", "p21", "RB1"]
for node in actives_cellcycle:
    maboss_cellcycle_model.network.set_istate(node, [0, 1])

inactives_cellcycle = ["CyclinB", "CyclinD", "E2F1"]
for node in inactives_cellcycle:
    maboss_cellcycle_model.network.set_istate(node, [1, 0])

outputs_cellcycle = ["CyclinB", "CyclinD", "E2F1", "Proliferation"]
maboss_cellcycle_model.network.set_output(outputs_cellcycle)

The result of the simulation are the following trajectories:

In [81]:
maboss_cellcycle_result = maboss_cellcycle_model.run()

maboss_cellcycle_result.get_nodes_probtraj().plot(
    legend=True,
    xlabel="Time (a.u.)",
    ylabel="Cell Cycle activation (node probabilities)"
)
plt.savefig("5.pdf")

## Mutants simulation
The model can be used to study the effect of mutations which activate or disable some nodes, or in general which deviate their functioning from physiological conditions.

### Single mutations
One case is the permanent inactivation of node "FOXA1". Except for the mutation, this simulation has the same initial conditions as the wild type model and all the output nodes are observed as phenotypes.

In [82]:
maboss_FOXA1_model = maboss_wtA_model.copy()

maboss_FOXA1_model.mutate("FOXA1", "OFF")

# MC debug experimenting.
maboss_FOXA1_model.network.set_istate("EGF", [0,1])
maboss_FOXA1_model.network.set_istate("FGF", [0,1])
maboss_FOXA1_model.network.set_istate("Nutrients", [0,1])
maboss_FOXA1_model.network.set_istate("SPOP", [0.5,0.5])

outputs_network = df_nodes_pathways.loc[df_nodes_pathways["pathway"] == "Output"]["node"].tolist()
maboss_FOXA1_model.network.set_output(outputs_network)

The results of this simulation are

In [83]:
maboss_FOXA1_result = maboss_FOXA1_model.run()

maboss_FOXA1_result.get_nodes_probtraj().plot(
    legend=True,
    xlabel="Time (a.u.)",
    ylabel="FOXA1 OFF - Phenotypes"
)
plt.savefig("6A.pdf")

Another mutation affecting a single node is the inactivation of gene TP53, which affects the inactivation of node "p53". Phenotype "Caspase3" is added to the outputs of the previous simulation.

In [84]:
maboss_p53_model = maboss_wtA_model.copy()

maboss_p53_model.mutate("p53", "OFF")

outputs_p53 = outputs_network.copy()
outputs_p53.append("Caspase3")
maboss_p53_model.network.set_output(outputs_p53)

The result of the simulation is

In [85]:
maboss_p53_result = maboss_p53_model.run()

maboss_p53_result.get_nodes_probtraj().plot(
    legend=True,
    xlabel="Time (a.u.)",
    ylabel="TP53 OFF - Phenotypes"
)
plt.savefig("6B.pdf")

### Multiple mutations
During cancer progression multiple mutations accumulate and their effects and interactions can be simulated.

An example is the interaction between gene fusion TMPRSS2:ERG and a loss-of-function mutation of gene NKX3_1. The starting model is the wild type model, with active nodes in list `actives_multiple` and random value for node "SPOP" as initial conditions. Outputs of the simulation are the output nodes of the network. Then mutations are applied simultaneously.

In [87]:
maboss_multiple_model = maboss_wtA_model.copy()

actives_multiple = ["EGF", "FGF", "Nutrients", "Carcinogen"]
for node in actives_multiple:
    maboss_multiple_model.network.set_istate(node, [0, 1])
maboss_multiple_model.network.set_istate("SPOP", [0.5, 0.5])

outputs_network.append("AR_ERG")  # MC debug experimenting.
maboss_multiple_model.network.set_output(outputs_network)

maboss_multiple_model.mutate("AR_ERG", "ON")
#maboss_multiple_model.network.set_istate("AR_ERG", [0, 1])  # MC debug experimenting.
maboss_multiple_model.mutate("NKX3_1", "OFF")

The result of the simulation is

In [88]:
maboss_multiple_result = maboss_multiple_model.run()

maboss_multiple_result.get_nodes_probtraj().plot(
    legend=True,
    xlabel="Time (a.u.)",
    ylabel="AR_ERG ON / NXK3-1 - Phenotypes"
)
plt.savefig("7.pdf")

In [89]:
# MC debug experimenting.
display(df_nodes_interactions.info())
display(df_nodes_interactions.loc[
    (df_nodes_interactions["Target node"] == "Metastasis") |
    (df_nodes_interactions["Target node"] == "Migration") |
    (df_nodes_interactions["Target node"] == "Invasion")
])

# Personalisation of the prostate Boolean model
The wild type model can be integrated with data derived from experiments and existing databases, to create personalised models for patient-specific analysis and treatments. Authors follow their PROFILE methodology using bulk omics data.

## Phenotype distribution of TCGA patients